# In-Painting Application

In this small project, the foreground/background of an image is automatically segmented and can be replaced by a generated image, conditioned by a text prompt.

- [SAM](#) is used as segmentation model.
- [Stable Diffusion XL Turbo](#) is used as in-painting model.
- [Gradio](#) is used as web UI engine.

As you can see, all models and tools come from [HuggingFace](#).

Here's an example:

TBD.

Table of contents:

- [Setup](#setup)
- [Zero-Shot Image Segmenation with SAM](#zero-shot-image-segmenation-with-sam)

## Setup

In [1]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor
from diffusers import DiffusionPipeline, AutoPipelineForText2Image, AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

import torch
import numpy as np

In [2]:
# Name of each GPU
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

0 NVIDIA GeForce RTX 3060
1 NVIDIA T500


In [3]:
# Select GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
!nvidia-smi

Sat Dec  6 11:58:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA T500                    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8            N/A  / 5001W |       5MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Zero-Shot Image Segmenation with SAM

[SAM](#) is ...

In [ ]:
# Load SAM processor + model and move to GPU
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]